## CCXT 是什么

ccxt 是 github上最火热的加密货币交易库， 包含了近百个交易所api。支持 Python、JavaScript、PHP 三种语言调用。

我用python来调用ccxt, 数据分析，策略设计和信号生成。

安装ccxt：

pip3 install ccxt

ccxt官网：

https://ccxt.readthedocs.io/en/latest/manual.html

ccxt的官方网站，同时也是它的技术文档，内容大而全，包含了所有的技术细节。

但是我们并不用通读它才能使用ccxt。

对我来说，掌握基本概念了后，就可以开始根据自己的需要调用ccxt的各种命令，来为自己的项目服务了。
例如我们这里的第一个项目就是在python环境下调用ccxt的命令来抓取交易所的行情数据。
对于以项目为导向为目的的学习，我非常赞同“不求甚解”的态度，先让代码跑起来，至于为什么是用这个命令，还有没有更简洁有效的方法，我们先不考虑。不然太容易浪费时间。

运行下面的代码，我们可以币安交易所下载过去100天所有加密货币交易对的k线数据：开盘收盘价，最高价最低价以及交易量。
如果只是需要数据，那拿走下面的代码就行了。

如果要深入理解一点代码，以达到修改代码满足自己其他需求，可以继续看我下面的代码讲解。

In [ ]:
import numpy as np
import pandas as pd
import ccxt
import time
import dateutil
from datetime import datetime

exchange = ccxt.binance()

all_coins = list(exchange.load_markets().keys())

all_candles = []

for symbol in all_coins:
    df = pd.DataFrame(exchange.fetch_ohlcv(symbol, timeframe='1d', limit=1000))
    df['symbol'] = symbol
    all_candles.append(df)
    
all_candles = pd.concat(all_candles)

all_candles.columns = ['Datetime', 'Open', 'High', 'Low', 'Close', 'Vol', 'Symbol']

all_candles['Datetime'] = all_candles['Datetime'].apply(
   lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(x / 1000.)))

all_candles.to_csv("binance_coins_daily_candlesticks.csv")

首先，最前面几行含有`import` 的代码，是来用载入我们后面程序所需要的第三方库，例如ccxt。
这些第三方库一般需要通过 `pip` 命令来安装。如何安装第三方库，可以参考上一篇的内容。注意一定安装在我们想要安装的该项目的虚拟环境里。

`exchange = ccxt.binance()` 是将币安设定为我们的默认交易所，并给他一个名字exchange,你也可以叫他bian（拼音），都没有关系，exchange的作用就是方便我们后面调用它里面的命令。

当然ccxt也支持其他交易所，使用`ccxt.exchanges`可以查看它所支持的所有交易所，基本上有点规模的都会支持。
如果要调用其他交易所的数据，我们把`exchange = ccxt.binance()`里面的 `binance`换掉就行了。

`exchange.load_markets()` 返回交易所整个市场的情况，这里的数据是以字典（dictionary）的形式存在的，如果不了解python里面字典是什么，可以参考：https://www.liaoxuefeng.com/wiki/1016959663602400/1017104324028448 

这里我们感兴趣的事这个字典数据里面的key(键)，因为它们是加密货币对的名字。 我们用 `exchange.load_markets().keys()`把这些加密货币对的名字提取出来，为了看起来顺眼，外面套一个1`list`的命令，把它们装在列表里，然后命名为 `all_coins`。

接下来`all_candles = []` 是建了一个空白的列表，名字叫做`all_candles`，目的是装后面我们抓到的数据。

下来这个for循环，对于没有接触过任何编程的人来说，可能稍微费点脑子。

我来掰开揉碎。从里面开始，`exchange.fetch_ohlcv(symbol, timeframe='1d', limit=1000)`。

`exchange.fetch_ohlcv`是抓取交易所数据的核心命令，其他的代码都是围绕这个命令来写的。
这个命令里面有几个必须的参数：

  `symbol` 是交易对，比如BTC/USDT。
  `timeframe='1d'`  是k线数据周期，这里1d是一天，可以是1h, 4h, 15m,等等。在ccxt文档搜索timeframe可以找到所支持的所有周期。
  `limit=1000` 是我们想要获取的到当前时间的k线的数量，最大值是1000.

我们可以任意尝试修改里面的参数来获得数据 `exchange.fetch_ohlcv(symbol, timeframe='1d', limit=1000)`。

现在数据有了，但是看起来不太方便，也不利于后面的统计分析，所以我们把上面的命令套在 `pd.DataFrame()`里面，这样数据就漂亮多了。

赋值命名为 `df`。 这里`pd`就是我们一开始导入的一个第三方库`pandas`的简写，后面跟的是 `DataFrame` 这个命令，用来把数据装到数据框里。

根据ccxt的文档说明，第一列是时间，紧跟着是 'Open', 'High', 'Low', 'Close', 'Vol'。后面我们会把更多的不同的加密货币对行情数据叠放在一起，所以我们要添加新的一列数据来标注这一行是那个加密货币对。 `df['symbol'] = symbol`

现在我们需要给每一列的数据添加名称：

`df.columns = ['Datetime', 'Open', 'High', 'Low', 'Close', 'Vol', 'Symbol']`

注意到我们数据的第一行的时间很奇怪，不是我们经常看到的样子。
我们用下面这一行命令来转换：

`df['Datetime'] = df['Datetime'].apply(
   lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(x / 1000.)))`
   
看起来有点复杂，不要被吓到，这里要发挥我们“不求甚解”的精神，明白大意即可。
    该行代码把 `Datetime` 这一列时间数据转换成 年月日，时分秒的格式。

这是一个币的数据，我们想要所有币的数据，这时候就要用到for循环。

for 循环就是把我们上面的一通操作运用到所有的加密货币对上。

还记得我们的`all_coins = list(exchange.load_markets().keys())`吗？

我们要抓取这里面所有的加密货币对的k线数据。

In [ ]:
all_candles = []

for symbol in all_coins:
    
    df = pd.DataFrame(exchange.fetch_ohlcv(symbol, timeframe='1d', limit=10))
    df['symbol'] = symbol
    
    df.columns = ['Datetime', 'Open', 'High', 'Low', 'Close', 'Vol', 'Symbol']

    df['Datetime'] = df['Datetime'].apply(
    lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(x / 1000.)))
   
    all_candles.append(df)

In [ ]:
all_candles = pd.concat(all_candles)

In [ ]:
all_candles.to_csv("binance_coins_daily_candlesticks.csv")